
# Stock Analysis using OpenAI Functions and LangChain Multi Agent

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

**Diclaimer**: This is only a demo notebook of stocks analysis. The recommendations or advise may not be used for actual invesment decision.

In [1]:
!pip install -q yfinance langchain openai langchain_openai langchainhub langsmith httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00


In [7]:
import openai
import json
import os
import numpy as np
import yfinance as yf
from langchain_core.tools import StructuredTool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from getpass import getpass

## Defining a custom method for obtaining stats of a stock

**yfinance** is an open source library to access the financial data available on Yahoo Finance, which offers an excellent range of market data on stocks, bonds and currencies along with market news, reports and analysis.

- Define a function which returns total gains, average and standard deviation of daily prices changes of stocks for a specified number of months

In [8]:
def get_stock_stats(ticker: str):

    '''
        input string ticker is The correct ticker symbol for the stock in Nation Stock Exchange (NSE) India.
    '''

    # Get ticker name correctly
    msft = yf.Ticker(ticker.split(".")[0] + ".NS")

    # get historical market data
    hist = msft.history(period = str(3) + "mo" )

    # Compute the market data
    hist['daily_changes']  = (hist['Close'] - hist['Open']) * 100 / hist['Open']

    # Compute different statistics
    total_gain = (hist.iloc[-1]["Close"] - hist.iloc[0]["Open"] ) * 100 / hist.iloc[0]["Open"]
    avg_daily_changes = np.mean(hist['daily_changes'])
    std_daily_changes = np.std(hist['daily_changes'])

    stock_stats = {'total_gain_in_percentage': round(total_gain, 3),
                   'average_daily_changes_in_percentage': round(avg_daily_changes, 3),
                   'std_daily_changes_in_percentage': round(std_daily_changes, 3)}

    return json.dumps(stock_stats)

In [9]:
get_stock_stats("TATAPOWER")

'{"total_gain_in_percentage": -19.18, "average_daily_changes_in_percentage": -0.567, "std_daily_changes_in_percentage": 1.716}'

## Configuring the function as a tool

## Configuring tool as OpenAI function

In [10]:
os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY')

OPENAI_API_KEY··········


In [ ]:
#os.environ["LANGCHAIN_API_KEY"] = getpass("Langchain API Key:")

Langchain API Key:··········


In [11]:
tool = StructuredTool.from_function(get_stock_stats)


In [12]:
llm = ChatOpenAI(model_name='gpt-4o-2024-08-06',
             temperature=0.0,
             max_tokens = 2048)

In [13]:
tools = [
    tool
]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.invoke( {"input": "How to calculate risk of a stock?"} )



> Entering new AgentExecutor chain...
Calculating the risk of a stock typically involves assessing its volatility and potential for loss. Here are some common methods to evaluate stock risk:

1. **Standard Deviation**: 
   - This measures the stock's price volatility. A higher standard deviation indicates a higher risk as the stock price is more spread out over a range of values.

2. **Beta**:
   - Beta measures a stock's volatility relative to the overall market. A beta greater than 1 indicates that the stock is more volatile than the market, while a beta less than 1 indicates less volatility.

3. **Value at Risk (VaR)**:
   - VaR estimates the maximum potential loss over a specific time frame with a given confidence level. It helps in understanding the potential downside risk.

4. **Sharpe Ratio**:
   - This ratio measures the risk-adjusted return of a stock. It is calculated by subtracting the risk-free rate from the stock's return and dividing by the standard deviation. A higher 

{'input': 'How to calculate risk of a stock?',
 'output': "Calculating the risk of a stock typically involves assessing its volatility and potential for loss. Here are some common methods to evaluate stock risk:\n\n1. **Standard Deviation**: \n   - This measures the stock's price volatility. A higher standard deviation indicates a higher risk as the stock price is more spread out over a range of values.\n\n2. **Beta**:\n   - Beta measures a stock's volatility relative to the overall market. A beta greater than 1 indicates that the stock is more volatile than the market, while a beta less than 1 indicates less volatility.\n\n3. **Value at Risk (VaR)**:\n   - VaR estimates the maximum potential loss over a specific time frame with a given confidence level. It helps in understanding the potential downside risk.\n\n4. **Sharpe Ratio**:\n   - This ratio measures the risk-adjusted return of a stock. It is calculated by subtracting the risk-free rate from the stock's return and dividing by th

In [17]:
agent_executor.invoke( {"input": "What is the return from TATAPOWER stock in NSE in last month?"} )



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`


{"total_gain_in_percentage": -19.18, "average_daily_changes_in_percentage": -0.567, "std_daily_changes_in_percentage": 1.716}The return from TATAPOWER stock in the NSE over the last month is -19.18%.

> Finished chain.


{'input': 'What is the return from TATAPOWER stock in NSE in last month?',
 'output': 'The return from TATAPOWER stock in the NSE over the last month is -19.18%.'}

## OpenAI Multi Function Agent

In [18]:
response = agent_executor.invoke( {"input":
    "Between tatapower and adanipower in NSE India, which stock had \
    observerd higher volatility in last one month? Explain the steps for your response. \
    The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary."}
)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`
responded: To determine which stock had higher volatility between Tata Power and Adani Power in the last month on the NSE India, we need to follow these steps:

1. **Identify the Ticker Symbols**: 
   - Tata Power's ticker symbol on NSE is "TATAPOWER".
   - Adani Power's ticker symbol on NSE is "ADANIPOWER".

2. **Fetch Stock Statistics**:
   - We will use the `get_stock_stats` function to fetch the stock statistics for both "TATAPOWER" and "ADANIPOWER".

3. **Analyze Volatility**:
   - Volatility is typically measured by the standard deviation of the stock's returns over a specific period. We will compare the volatility of both stocks based on the data retrieved.

4. **Compare and Conclude**:
   - Compare the volatility values of both stocks to determine which one had higher volatility.

Let's start by fetching the stock statistics for both Tata Power and Adani Power.

{"total_gain_in_p

In [19]:
import textwrap

In [20]:
wrapped_text = textwrap.fill(response['output'],
                             width=120,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Based on the stock statistics retrieved for the last month:

- **Tata Power (TATAPOWER)**:
  - Standard deviation of
daily changes: 1.716%

- **Adani Power (ADANIPOWER)**:
  - Standard deviation of daily changes: 4.109%

### Conclusion:
The standard deviation of daily changes is a measure of volatility. Adani Power (ADANIPOWER) had a higher standard
deviation of daily changes (4.109%) compared to Tata Power (TATAPOWER) which had a standard deviation of 1.716%.
Therefore, Adani Power observed higher volatility in the last month compared to Tata Power.
